# Playground for strokes graph

In [ ]:
# import libraries
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import networkx as nx
import folium

Import example data set

In [ ]:
# read in toy graph (Bubenec)
streets = gpd.read_file(momepy.datasets.get_path("bubenec"), layer="streets")
streets = momepy.remove_false_nodes(streets)
# streets["id"] = streets.index

# ax = streets.plot(figsize=(8, 8), column="id", cmap = "Accent")
# ax.set_axis_off()
# ax.legend()

Workflow (will be wrapped in function later)

- [ ] input is a set of linestrings
- [ ] remove false nodes 
- [ ] convert linestrings to primal graph
- [ ] get points and lines gdf from primal graph
- [ ] run COINS on lines gdf to find strokes
- [ ] add mapping of strokeID:edgeIDs to stroke gdf (use momepy's **edge ID**, not indexing)
- [ ] add stroke attribute to each edge on primal graph

In [ ]:
# define variable defaults (will be arguments passed to future function)
angle_threshold=0
flow_mode=False

In [ ]:
# remove false nodes
streets = momepy.remove_false_nodes(streets)

# add unique edge ID to streets, already HERE!
streets["edge_id"] = streets.index

# make primal graph
graph = momepy.gdf_to_nx(streets, approach="primal")

# get gdfs of points and lines
points, lines = momepy.nx_to_gdf(graph, points=True, lines=True)

# make coins
coins = momepy.COINS(lines, angle_threshold=angle_threshold, flow_mode=flow_mode)

# get gdfs from COINS class
stroke_attribute = coins.stroke_attribute()
stroke_gdf = coins.stroke_gdf()

# add stroke_id column
stroke_gdf["stroke_id"] = stroke_gdf.index

# add edge_ids column (using COINS.stroke_attribute to map into ID defined in lines gdf)
stroke_gdf["edge_ids"] = stroke_gdf.stroke_id.apply(
    lambda x: list(
        lines.iloc[
            stroke_attribute[stroke_attribute == x].index]["edge_id"]
    )
)

stroke_gdf.head()

In [ ]:
# make dictionary for primal graph, edge_id:edge_name
d = nx.get_edge_attributes(graph, "edge_id")
d = {v:k for k,v in d.items()}
d

In [ ]:
# for each edge, add "stroke_id" as attribute to graph
for _, row in stroke_gdf.iterrows():
    for e in row.edge_ids: 
        graph.edges[d[e]]["stroke_id"] = row.stroke_id

In [ ]:
d_strokes = nx.get_edge_attributes(graph, "stroke_id")
d_edges = nx.get_edge_attributes(graph, "edge_id")

In [ ]:
edges_strokes = {d_edges[k]:d_strokes[k] for k in d_edges} 
edges_strokes

In [ ]:
# m = stroke_gdf.explore(tiles="cartodb.positron", column = "stroke_id", name = "strokes", cmap = "Set2")
# lines.explore(m=m, column = "id", name = "lines")
# folium.LayerControl().add_to(m)
# m

***